# HOMEWORK 6

In this homework you are going to use the Harris corner detector to detect the corners of a document. Document detection is a crucial task for many applications, e.g., text recognition, automatic passport reading (at airport gates), etc.

You will also have to design your own feature descriptor in order to localize and distinguish among the 4 document corners.

At the end of this notebook, there are a couple of questions for you to answer.

So let's begin, shall we?

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

Let's load the image we will be working on in this homework.

In [ ]:
# Let's read the image
img = cv2.imread('data/document.jpg')
# Convert it to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)/255
rows, cols = gray.shape

# Let's plot the images (colour and gray scale)
plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(gray, cmap='gray')

### Harris Corner Detector
Let us now compute Harris corners. Remember that the Harris detector computes the "cornerness" score for each image pixel.

In [ ]:
# Compute Harris corners (use the available OpenCV functions)
# Suggested parameters:
#            block size of 2 pixels
#            gradient kernel size of 3 pixels
#            k parameter equal to 0.04
cornerness = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
# We are not interested in edges, so put to zero all negative cornerness values
cornerness[cornerness < 0] = 0

# Since cornerness has a huge dynamic range, let's take the logarithm for better visualization and manipulation
cornerness = np.log(cornerness + 1e-6)

In [ ]:
# Let's now plot the image and the corresponding Harris corners (in log scale)
plt.subplot(121), plt.imshow(img)
plt.subplot(122), plt.imshow(cornerness)

At this point, you can see that the Harris detector has detected a lot of features. Not only the four document corners, but also the corners corresponding to (black) letters printed on (white) paper. How can we filter out everything but the 4 document corners?

For that purpose, let's design a custom feature descriptor suitable to detect the document corners. In order to do so, let's have a look at the top left corner.

![Top-left corner](data/document_descriptor_example.jpg "Top-left corner")

A good descriptor of that corner, given a certain neghbouring region, would be to assess that the bottom-right quadrant is (much) brighter than the other three quadrants (i.e. top-left, top-right, bottom-left). Let's then implement it :-) I'll do the implementation for the top-left corner, you shall do the rest.

In [ ]:
# Detection thresholds
th_top_left, th_top_right = -1e6, -1e6
th_bottom_left, th_bottom_right = -1e6, -1e6

# Corner coordinates
opt_top_left, opt_top_right = None, None
opt_bottom_left, opt_bottom_right = None, None

# Size of each quadrant (in pixels)
quad_size = 7

# Let's now scan the Harris detection results
for r in range(quad_size, rows-quad_size):
    for c in range(quad_size, cols-quad_size):
        # Edges with too small cornerness score are discarded, -7 seems like a good value
        if cornerness[r, c] < -7:
            continue
        
        # Extract block consisting of 4 quadrants
        block = 255*gray[r-quad_size:r+quad_size+1, c-quad_size:c+quad_size+1]
        
        # Extract the four quandrants
        quad_top_left = block[0:quad_size, 0:quad_size]
        # quad_top_right = block[0:quad_size, quad_size+1:2*quad_size+1]
        # quad_bottom_left = block[quad_size+1:2*quad_size+1, 0:quad_size]
        # quad_bottom_right2 = block[quad_size+1:2*quad_size+1, quad_size+1:2*quad_size+1]
        quad_top_right = block[0:quad_size, quad_size+1:]
        quad_bottom_left = block[quad_size+1:, 0:quad_size]
        quad_bottom_right = block[quad_size+1:, quad_size+1:]

        # Top-left corner
        # For the top-left document corner, the bottom-right quadrant is mostly paper and the rest is
        # darker background. Therefore, I suggest the descriptor to be the average difference between
        # the paper quandrant and the sum of the 3 remaining bakcground quandrants
        descriptor_top_left = np.mean(quad_bottom_right) - np.mean(quad_top_left) - np.mean(quad_top_right) - np.mean(quad_bottom_left)
        # Let's detect the best descriptor
        if descriptor_top_left > th_top_left:
            # We update the threshold
            th_top_left = descriptor_top_left
            # And we update the optimal location
            opt_top_left = (c, r)
            
        # Top-right corner
        descriptor_top_right = np.mean(quad_bottom_left) - np.mean(quad_bottom_right) - np.mean(quad_top_left) - np.mean(quad_top_right)
        # Let's detect the best descriptor
        if descriptor_top_right > th_top_right:
            # We update the threshold
            th_top_right = descriptor_top_right
            # And we update the optimal location
            opt_top_right = (c, r)
            
        # Bottom-left corner
        descriptor_bottom_left = np.mean(quad_top_right) - np.mean(quad_bottom_left) - np.mean(quad_bottom_right) - np.mean(quad_top_left)
        # Let's detect the best descriptor
        if descriptor_bottom_left > th_bottom_left:
            # We update the threshold
            th_bottom_left = descriptor_bottom_left
            # And we update the optimal location
            opt_bottom_left = (c, r)
            
        # Bottom-right corner
        descriptor_bottom_right = np.mean(quad_top_left) - np.mean(quad_top_right) - np.mean(quad_bottom_left) - np.mean(quad_bottom_right)
        # Let's detect the best descriptor
        if descriptor_bottom_right > th_bottom_right:
            # We update the threshold
            th_bottom_right = descriptor_bottom_right
            # And we update the optimal location
            opt_bottom_right = (c, r)

# Let's draw circles at the detected corners
out = cv2.circle(img, opt_top_left, 3, (255,0,0), -1)
out = cv2.circle(img, opt_top_right, 3, (255,0,0), -1)
out = cv2.circle(img, opt_bottom_left, 3, (255,0,0), -1)
out = cv2.circle(img, opt_bottom_right, 3, (255,0,0), -1)

# And finally we plot the images (with the detected document corners)
plt.subplot(121), plt.imshow(out)
plt.subplot(122), plt.imshow(cornerness)

### Questions
* Does it matter whether the picture has been taken by a 1Mpx camera or a 12Mpx camera? How?

    The more megapixels in the camera that took the photo, the higher its resolution. Since the Harris algorithm is not scaling invariant, we may not find the keypoints we are interested in (corners).

* If we increased the resolution of the camera, what would you change in the current algorithm?

    Variant 1. Resize image to smoller resolution
    
    Variant 2. Increase cornerHarris params blockSize, ksize and increase quad_size

Remember, I am **not** looking for a particular answer. I want to see how you think, so be creative ;-)